In order to predict 2 out of 7 lottery numbers in Signapore's Toto, I used the dataset sourced from Lottolyzer, which gave the results from the games drawn between3rd July 2008 to 21st July 2022. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

In [2]:
df = pd.read_csv("Downloads/ToTo (210722).csv")
df.head()

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number,From Last,...,Division 3 Winners,Division 3 Prize,Division 4 Winners,Division 4 Prize,Division 5 Winners,Division 5 Prize,Division 6 Winners,Division 6 Prize,Division 7 Winners,Division 7 Prize
0,3784,2022-07-21,4,15,31,42,43,47,39,"43,47",...,184.0,1310.0,341.0,386.0,6789.0,50.0,8114.0,25.0,119981.0,10.0
1,3783,2022-07-18,3,20,36,41,43,47,9,36,...,84.0,2017.0,194.0,477.0,3996.0,50.0,6194.0,25.0,73762.0,10.0
2,3782,2022-07-14,8,10,15,36,39,44,9,NaN,...,554.0,1515.0,1438.0,319.0,26324.0,50.0,39554.0,25.0,471689.0,10.0
3,3781,2022-07-11,1,6,13,25,34,43,42,NaN,...,218.0,1918.0,613.0,372.0,13650.0,50.0,16609.0,25.0,240867.0,10.0
4,3780,2022-07-07,16,17,20,33,39,49,7,NaN,...,107.0,2184.0,328.0,389.0,5880.0,50.0,10089.0,25.0,112032.0,10.0


In [3]:
df = pd.DataFrame(df)
df2 = df.iloc[:790,2:9] 
### Removing data before Oct 7, 2014 draw as there was change from "6 out of 45" to "6 out of 49" numbers format. 
### Also extracting only the "numbers" columns 
df2

,Winning Number 1,2,3,4,5,6,Additional Number
0,4,15,31,42,43,47,39
1,3,20,36,41,43,47,9
2,8,10,15,36,39,44,9
3,1,6,13,25,34,43,42
4,16,17,20,33,39,49,7
...,...,...,...,...,...,...,...
785,5,22,25,28,33,34,32
786,8,11,17,21,41,49,35
787,4,9,31,34,40,44,35
788,19,28,31,43,45,48,41


In [4]:
scaler = StandardScaler().fit(df2.values)
transformed_dataset = scaler.transform(df2.values)

transformed_df = pd.DataFrame(data=transformed_dataset, index=df2.index)

transformed_df.head()

,0,1,2,3,4,5,6
0,-0.508958,0.128221,1.229488,1.666873,1.015412,0.727009,0.951736
1,-0.688536,0.809881,1.854357,1.543041,1.015412,0.727009,-1.159275
2,0.209357,-0.553439,-0.770091,0.923881,0.469807,0.180368,-1.159275
3,-1.047694,-1.098766,-1.020039,-0.438271,-0.212199,-0.001845,1.162837
4,1.645986,0.400885,-0.145223,0.552385,0.469807,1.091436,-1.300009


In [5]:
number_of_rows = df2.values.shape[0]
print('Number of Games:', number_of_rows)

window_length = 7
number_of_features = df2.values.shape[1]
print(number_of_features)

X = np.empty([ number_of_rows-window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

print(X.shape)
print(y.shape)

print(X[0])

Number of Games: 790
7
(783, 7, 7)
(783, 7)
[[-5.08957642e-01  1.28221049e-01  1.22948836e+00  1.66687264e+00
   1.01541195e+00  7.27008810e-01  9.51736375e-01]
 [-6.88536265e-01  8.09880731e-01  1.85435699e+00  1.54304065e+00
   1.01541195e+00  7.27008810e-01 -1.15927458e+00]
 [ 2.09356850e-01 -5.53438633e-01 -7.70091268e-01  9.23880697e-01
   4.69807164e-01  1.80368303e-01 -1.15927458e+00]
 [-1.04769351e+00 -1.09876638e+00 -1.02003872e+00 -4.38271196e-01
  -2.12198825e-01 -1.84520003e-03  1.16283747e+00]
 [ 1.64598584e+00  4.00884922e-01 -1.45222634e-01  5.52384726e-01
   4.69807164e-01  1.09143582e+00 -1.30000865e+00]
 [-8.68114888e-01 -1.50776219e+00 -1.51993363e+00 -1.90607215e-01
  -1.03060601e+00 -9.12912713e-01 -7.37072392e-01]
 [-8.68114888e-01  1.28221049e-01 -5.20143814e-01 -1.90607215e-01
   7.42609559e-01  9.09222313e-01  8.81369343e-01]]


In [6]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam


In [7]:
model2 = Sequential()

model2.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model2.add(Dropout(0.2))
model2.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model2.add(Dropout(0.2))
model2.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model2.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

model2.add(Dense(59))
model2.add(Dense(number_of_features))

model2.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

model2.fit(x=X, y=y, batch_size=32, epochs=500, verbose=2)

Epoch 1/500
25/25 - 24s - loss: 1.0001 - accuracy: 0.1622 - 24s/epoch - 957ms/step
Epoch 2/500
25/25 - 12s - loss: 0.9937 - accuracy: 0.1903 - 12s/epoch - 461ms/step
Epoch 3/500
25/25 - 14s - loss: 0.9924 - accuracy: 0.1928 - 14s/epoch - 558ms/step
Epoch 4/500
25/25 - 14s - loss: 0.9883 - accuracy: 0.1980 - 14s/epoch - 567ms/step
Epoch 5/500
25/25 - 14s - loss: 0.9874 - accuracy: 0.2056 - 14s/epoch - 543ms/step
Epoch 6/500
25/25 - 12s - loss: 0.9857 - accuracy: 0.2043 - 12s/epoch - 490ms/step
Epoch 7/500
25/25 - 13s - loss: 0.9846 - accuracy: 0.2005 - 13s/epoch - 512ms/step
Epoch 8/500
25/25 - 14s - loss: 0.9822 - accuracy: 0.2031 - 14s/epoch - 564ms/step
Epoch 9/500
25/25 - 15s - loss: 0.9810 - accuracy: 0.1954 - 15s/epoch - 607ms/step
Epoch 10/500
25/25 - 18s - loss: 0.9794 - accuracy: 0.2031 - 18s/epoch - 715ms/step
Epoch 11/500
25/25 - 17s - loss: 0.9805 - accuracy: 0.1954 - 17s/epoch - 672ms/step
Epoch 12/500
25/25 - 14s - loss: 0.9759 - accuracy: 0.2043 - 14s/epoch - 564ms/step
E

In [8]:
to_predict = df2.iloc[:7]
to_predict = np.array(to_predict)
scaled_to_predict = scaler.transform(to_predict)
y_pred = model2.predict(np.array([scaled_to_predict]))

1/1 [==============================] - 6s 6s/step


In [9]:
print('The predicted numbers in the 25th July lottery draw are:', scaler.inverse_transform(y_pred).astype(int))


The predicted numbers in the 25th July lottery draw are: [[21 24 26 31 34 37 41]]


The actual numbers in the 25th July lottery draw are: 2, 6, 29, 30, 31, 44, 40 . 

Managed to predict 2 out of 7 numbers - 31 & 40. 
